# Gather insights from industrial data stores

## Prerequisites

Before getting started, make sure you have the following:

- Python 3.x installed
- Access to an AWS account with permissions to use Amazon Bedrock and access to Anthropic Claude 2.1 and Claude 3.
- The following repo: `https://github.com/aws-samples/industrial-data-store-simulation-chatbot` with this notebook in the same project directory
- (recommended) If running on Amazon SageMaker Studio Jupyter Lab, a `ml.t3.medium` is recommended. This is the default instance type for CPU-based SageMaker images, and is available as part of the [AWS Free Tier](https://aws.amazon.com/free)

## Introduction

In this notebook, you will get a better understanding of how the Chatbot application works under the hood and how it interacts with the database that simulates an MES. You will then be able to leverage these learnings to access other industrial data stores and build your own LLM-powered application

### Context

Large language models (LLMs) have good reasoning skills and are able to execute multi-step tasks, but to be useful in industrial applications, they require access to your data. LLMs are not designed to analyzed data by being directly fed information from a database. Instead, one can leverage the reasoning and code generation ability of the foundation model to come up with a plan to answer the question. Then, it can write code to retrieve this information from the database and finally use the information retrieved to answer the question.

This allows users that would not have previously been able to access this data because of the pre-existing requirement of not only being able to write SQL queries, but also needing to understand the database metadata and table relationship. Additionally, data now becomes accessible in a human-friendly chat interface instead of having to access a specific tool such as a proprietary client tool. 

### Interact with a Manufacturing Execution System (MES)
In this workshop, you will be creating a simulation Manufacturing Execution System (MES) represented as a relational database. MES are designed to facilitate the coordination and optimization of the production process, tracking key components of a manufacturing execution system.

They typically work closely with other industrial systems such as ERP (where work orders will come from), SCADA or other control software (where the process is controlled and operational time-series data from PLCs are generated) and Data Historians (where operational time-series data from machines and analytics are stored). 

For simplicity sake, the simulation MES is a self-contained system, but the same concept can be extended to interact with several systems by creating agents that connect to those various systems. It can also be applied to other industrial data stores. We will dive deeper into this topic below

## Environment Setup
> Note: This is only required if you are running this notebook outside of the workshop

Before we start, let's set up the environment by installing the required packages and loading the necessary environment variables.

First, let's install the required packages listed in the requirements.txt file:

In [11]:
#install requirements
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Next, we'll load the environment variables from the .env file using the dotenv package:
> Note: This is not required when running this notebook in Amazon Sagemaker Studio JupyterLab

In [12]:
# Environment setup
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()


True

## Create the boto3 client
Interaction with the Bedrock API is done via the AWS SDK for Python: boto3.

### Use different clients
The [`boto3` Python SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) provides different clients for Amazon Bedrock to perform different actions.  The actions for `InvokeModel` and `InvokeModelWithResponseStream` are supported by Amazon Bedrock Runtime, whereas other operations, such as `ListFoundationModels`, are handled via the Amazon Bedrock client.

To create the boto3 client, we need to provide the following information:

- `region_name`: The AWS region where the Bedrock service is located.
- `profile_name`: The AWS profile to use for authentication (optional, defaults to the default profile).
- `service_name`: The name of the Bedrock service (`'bedrock-runtime'`).
- `endpoint_url`: The endpoint URL for the Bedrock Runtime service, which includes the AWS region.

In [13]:
import boto3
import os

session = boto3.Session(
    region_name=os.getenv("AWS_REGION", "us-east-1"), profile_name=os.getenv("AWS_PROFILE")
)
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    endpoint_url='https://bedrock-runtime.'+os.getenv('AWS_REGION', 'us-east-1')+'.amazonaws.com',
    )

Let us test that we are able to access Claude 3 via Bedrock. If you receive an error for accessing the model, see [Model access](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html) to enable model access. 

In this code snippet, we:

1. Create a JSON payload (`body`) with the necessary parameters for invoking the model, such as `max_tokens`, `temperature`, and the input message.
2. Invoke the model using the `invoke_model` method of the `bedrock_client`, passing the JSON payload and the model ID.
3. Parse the response body and print the generated text.

Experiment with different prompt, by changing the value of `prompt`

In [14]:
import json

prompt = "Hello, world"

body = json.dumps({
    "max_tokens": 4096,
    "temperature": 0.5,
    "messages": [
        {"role": "user", "content": prompt}
    ],
    "anthropic_version": "bedrock-2023-05-31"
})

response = bedrock_client.invoke_model(body=body, modelId="anthropic.claude-3-haiku-20240307-v1:0")
response_body = json.loads(response.get("body").read())      
print(response_body['content'][0]['text'])

Hello! It's nice to meet you.


Great, we have validated that we can access Claude 3 Haiku in Bedrock. We called the model directly using `boto3`, but we can use frameworks such as Langchain to make it easy to swap between different models accessible in Bedrock with minimal code changes.

The `get_llm` function creates an LLM object for the Langchain conversational Bedrock agent. It takes the following parameters:

- `model_kwargs`: A dictionary of model keyword arguments to be passed to the Bedrock model.
- `model_id`: The ID of the model to use (defaults to `"anthropic.claude-3-haiku-20240307-v1:0"`).

The function creates a `boto3` session and client, and then based on the `model_id`, it creates either a `BedrockChat` or `Bedrock` object from the Langchain library.

In [15]:
from langchain_community.chat_models import BedrockChat
from langchain.llms.bedrock import Bedrock

def get_llm(model_kwargs, model_id="anthropic.claude-3-haiku-20240307-v1:0"):
    """Creates the LLM object for the langchain conversational bedrock agent.
    Parameters:
        model_kwargs (dict): Dictionary of model_kwargs to be passed to the Bedrock model.
    Returns:
        langchain.llms.bedrock.Bedrock: Bedrock model
    """
    session = boto3.Session(
        region_name=os.getenv("AWS_REGION", "us-east-1"), profile_name=os.getenv("AWS_PROFILE")
    )
    bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        endpoint_url='https://bedrock-runtime.'+os.getenv('AWS_REGION', 'us-east-1')+'.amazonaws.com',
        )
    if (model_id == "anthropic.claude-3-haiku-20240307-v1:0") or (model_id == "anthropic.claude-3-sonnet-20240229-v1:0"):
        llm = BedrockChat(
            client=bedrock_client,
            model_id=model_id,
            model_kwargs=model_kwargs)
    else:
        llm = Bedrock(
            client=bedrock_client,
            model_id=model_id,
            model_kwargs=model_kwargs)

    return llm

Finally, we test the `get_llm` function with two different models (Claude 2.1 and Claude 3 Haiku) to demonstrate how Langchain can abstract away the differences in their APIs and allow us to call the models in the same way.

We measure the execution time for each model and print the generated responses along with the execution times.

In [16]:
# Let us test with 2 different models that have different APIs to access to show that Langchain can help abstract this away and always call the model the same way

import time
llm = get_llm(model_id="anthropic.claude-v2:1", model_kwargs={"max_tokens_to_sample": 4096,"temperature": 0.1})
llm_2 = get_llm(model_id="anthropic.claude-3-haiku-20240307-v1:0", model_kwargs={"max_tokens": 4096,"temperature": 0.1})

# For Claude 2.1
start_time = time.time()
response = llm.invoke("Tell me about the top 3 trends in Industrial Manufacturing")
end_time = time.time()
execution_time = end_time - start_time
print("Claude 2.1\n\n" + response)
print(f"\nExecution time: {execution_time:.2f} seconds\n\n")

# For Claude 3 Haiku
start_time2 = time.time()
response2 = llm_2.invoke("Tell me about the top 3 trends in Industrial Manufacturing")
end_time2 = time.time()
execution_time2 = end_time2 - start_time2
print("\n\nClaude 3 Haiku \n\n" + response2.content)
print(f"\nExecution time: {execution_time2:.2f} seconds")


Claude 2.1

 Here are three of the top trends in industrial manufacturing currently:

1. Smart manufacturing and Industry 4.0. This involves using advanced technologies like AI, automation, internet of things (IoT), cloud computing, etc. to enable intelligent and connected manufacturing systems. It allows for more data-driven, flexible, and efficient manufacturing operations.

2. Sustainable manufacturing. Manufacturers are focusing more on sustainability initiatives related to energy efficiency, reducing waste, recycling, responsible sourcing of materials, etc. This is being driven by both regulations and consumer expectations around environmental impact.

3. Advanced materials and additive manufacturing. There are new advanced materials like composites, nanomaterials, smart textiles etc. enabling lighter, stronger and more functional product designs. Additive manufacturing (3D printing) allows faster prototyping and creation of more complex geometries. This enables mass customization

## Database Exploration (OPTIONAL)
As discussed above, this code repository contains python code to generate a simulated MES. `MES-synthetic-data/sqlite-synthetic-mes-data.py` contains the code required to create a SQLite database with 6 tables, and then leverages `Faker` to create most of the simulated data, on-demand.

A copy of this function is available in the following cell with additional comments.

The `create_commands` is used to create the tables. These tables are a simplified representation of the tables that would normally be found in a Manufacturing Execution System. 


In [17]:
# Create the database and populate it with synthetic data
import sqlite3
import json
import os
from faker import Faker
import random
from sqlalchemy import create_engine, MetaData, Table, insert, select
from sqlalchemy.orm import sessionmaker

# Initialize Faker. Faker is used to create realistic simulation data
fake = Faker()

# Database settings
db_file = 'mes.db'

# Create SQLite database connection
conn = sqlite3.connect(db_file)
conn.execute("PRAGMA foreign_keys = ON")  # Enable foreign key constraints
cursor = conn.cursor()

# SQL commands to create tables
create_commands = (
    """
    CREATE TABLE IF NOT EXISTS Products (
        ProductID INTEGER PRIMARY KEY AUTOINCREMENT,
        Name TEXT NOT NULL,
        Description TEXT
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Machines (
        MachineID INTEGER PRIMARY KEY AUTOINCREMENT,
        Name TEXT NOT NULL,
        Type TEXT,
        Status TEXT CHECK (Status IN ('running', 'idle', 'maintenance'))
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS WorkOrders (
        OrderID INTEGER PRIMARY KEY AUTOINCREMENT,
        ProductID INTEGER NOT NULL,
        Quantity INTEGER NOT NULL CHECK (Quantity > 0),
        StartDate TEXT,
        EndDate TEXT,
        Status TEXT,
        FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Inventory (
        ItemID INTEGER PRIMARY KEY AUTOINCREMENT,
        Name TEXT NOT NULL,
        Quantity INTEGER NOT NULL CHECK (Quantity >= 0),
        ReorderLevel INTEGER NOT NULL CHECK (ReorderLevel >= 0)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS QualityControl (
        CheckID INTEGER PRIMARY KEY AUTOINCREMENT,
        OrderID INTEGER NOT NULL,
        Date TEXT NOT NULL,
        Result TEXT,
        Comments TEXT,
        FOREIGN KEY (OrderID) REFERENCES WorkOrders(OrderID)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS Employees (
        EmployeeID INTEGER PRIMARY KEY AUTOINCREMENT,
        Name TEXT NOT NULL,
        Role TEXT,
        Shift TEXT
    );
    """
)

# Create the tables based on the create_command above
def create_tables():
    try:
        for command in create_commands:
            cursor.execute(command)
        conn.commit()
        print("All tables created successfully")
    except Exception as e:
        conn.rollback()
        print(f"An error occurred: {e}")
    finally:
        cursor.close()
        conn.close()

# Load synthetically generated product information. This information is coming from the data_pools.json file
def load_data_pools(filename):
    with open(filename, 'r', encoding="utf-8") as file:
        data = json.load(file)
    return data

data_pools = load_data_pools('MES-synthetic-data/data_pools.json')
inventory_names = data_pools['inventory_names']
product_names = data_pools['product_names']
product_description = data_pools['product_descriptions']
qc_comments = data_pools['qc_comments']

# Insert synthetic data
def insert_data():
    create_tables()  # Create tables first

    # Create database engine
    engine = create_engine(f'sqlite:///{db_file}', echo=False)

    # Reflect the tables from the database
    metadata = MetaData()
    metadata.reflect(bind=engine)

    # Create a Session class bound to the engine
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Products generated from the data_pools
        products_table = metadata.tables['Products']
        for name, description in zip(product_names, product_description):
            session.execute(insert(products_table).values(Name=name, Description=description))
        session.commit()
        product_ids = fetch_product_ids(session, metadata)

        # Machines - the number of machines created in the DB can be modified
        machines_table = metadata.tables['Machines']
        for i in range(1, 6):  # Generating 5 machines
            session.execute(insert(machines_table).values(Name=f'Machine {i}', Type=random.choice(['Type A', 'Type B', 'Type C']), Status=random.choice(['running', 'idle', 'maintenance'])))
        session.commit()

        # Inventory
        inventory_table = metadata.tables['Inventory']
        for name in inventory_names:
            session.execute(insert(inventory_table).values(Name=name, Quantity=random.randint(0, 1000), ReorderLevel=random.randint(10, 100)))
        session.commit()

        # Work Orders - Dates leverage Faker to generate realistic values.
        # Note that the Products table had to be created first since the work orders are dependent on the productID
        work_orders_table = metadata.tables['WorkOrders']
        for _ in range(20):  # 20 work orders
            product_id = random.choice(product_ids)
            session.execute(insert(work_orders_table).values(
                ProductID=product_id,
                Quantity=random.randint(1, 100),
                StartDate=fake.date_between(start_date='-1y', end_date='today').isoformat(),
                EndDate=fake.date_between(start_date='today', end_date='+1y').isoformat(),
                Status=random.choice(['pending', 'in progress', 'completed', 'cancelled'])
            ))
        session.commit()
        order_ids = fetch_order_ids(session, metadata)

        # Employees - Here again, we are using Faker to generate realistic employee names
        employees_table = metadata.tables['Employees']
        for _ in range(10):  # 10 employees
            session.execute(insert(employees_table).values(
                Name=fake.name(),
                Role=random.choice(['Operator', 'Technician', 'Manager']),
                Shift=random.choice(['morning', 'evening', 'night'])
            ))
        session.commit()

        # Quality Control
        # Notice here how we are also dependent on work_orders_table to create quality records tied to the work orders
        # Since quality check entries will always be in the past, we are using Faker to generate realistic dates.
        quality_control_table = metadata.tables['QualityControl']
        for _ in range(20):
            session.execute(insert(quality_control_table).values(
                OrderID=random.choice(order_ids),
                Date=fake.date_between(start_date='-1y', end_date='today').isoformat(),
                Result=random.choice(['pass', 'fail', 'rework']),
                Comments=random.choice(qc_comments)
            ))
        session.commit()

    except Exception as e:
        session.rollback()
        print(f"An error occurred: {e}")
    finally:
        session.close()

# Helper functions for fetching IDs
def fetch_product_ids(session, metadata):
    products_table = metadata.tables['Products']
    return [id[0] for id in session.execute(select(products_table.c.ProductID)).fetchall()]

def fetch_order_ids(session, metadata):
    work_orders_table = metadata.tables['WorkOrders']
    return [id[0] for id in session.execute(select(work_orders_table.c.OrderID)).fetchall()]

In this example, the simulated MES database contains 6 tables. 

#### Products Table

- **Purpose**: Stores information about the products being manufactured.
- **Columns**:
  - `ProductID`: A unique identifier for each product, automatically generated.
  - `Name`: The name of the product, which is a required field.
  - `Description`: A text description of the product, which is optional.

#### Machines Table

- **Purpose**: Contains details about the machinery used in the manufacturing process.
- **Columns**:
  - `MachineID`: The unique identifier for each machine, automatically generated.
  - `Name`: The name of the machine.
  - `Type`: The type or category of the machine.
  - `Status`: The current status of the machine, restricted to 'running', 'idle', or 'maintenance' through a check constraint.

#### Work Orders Table

- **Purpose**: Tracks production work orders.
- **Columns**:
  - `OrderID`: A unique identifier for each work order, automatically generated.
  - `ProductID`: A reference to the `ProductID` in the Products table.
  - `Quantity`: The number of units to be produced, must be greater than 0.
  - `StartDate`: The start date of the work order.
  - `EndDate`: The expected end date of the work order.
  - `Status`: The current status of the work order.

#### Inventory Table

- **Purpose**: Manages inventory items, including materials or components.
- **Columns**:
  - `ItemID`: A unique identifier for each inventory item, automatically generated.
  - `Name`: The name of the inventory item.
  - `Quantity`: The current quantity in stock, must be non-negative.
  - `ReorderLevel`: The quantity at which more of the item should be ordered, also must be non-negative.

#### Quality Control Table

- **Purpose**: Records the outcomes of quality control checks for work orders.
- **Columns**:
  - `CheckID`: A unique identifier for each quality control check, automatically generated.
  - `OrderID`: A reference to the `OrderID` in the Work Orders table.
  - `Date`: The date when the quality control check was performed.
  - `Result`: The result of the quality control check.
  - `Comments`: Optional comments about the check.

#### Employees Table

- **Purpose**: Stores details about employees involved in the manufacturing process.
- **Columns**:
  - `EmployeeID`: A unique identifier for each employee, automatically generated.
  - `Name`: The name of the employee.
  - `Role`: The role or job title of the employee.
  - `Shift`: The work shift of the employee.


The database is created in the root of the project directory `mes.db`. Let's explore what we have created.

Since we create a SQLite database, we can use `sqlite3` to execute queries and explore the content of the simulated system. 

### Exploring the Simulated MES Database
--------------------------------------
The database is created in the root of the project directory `mes.db`. Let's explore what we have created.
Since we create a SQLite database, we can use `sqlite3` to execute queries and explore the content of the simulated system and store the results in pandas DataFrames. 


### Instructions:
1. Run the code above to explore the simulated MES database.
2. Observe the output, which displays the list of tables and the first 5 rows from each table.
3. Modify the code to execute different SQL queries or explore the data in different ways.
4. Use the `execute_query` function to run your custom SQL queries and retrieve the results as a pandas DataFrame.
5. Feel free to experiment with the code and explore the database further.

In [18]:
# Import required libraries
import sqlite3
import pandas as pd


# Function to execute a SQL query and return the results as a pandas DataFrame
def query_sqlite(query, db_path):
    """Creates a connection to a SQLite database and executes a query
    Parameters
    ----------
    query :
        An string containing SQL code
    db_path :
        Path to the SQLite database file
    Returns
    ----------
    pandas.DataFrame :
        the results of the SQL query
    """
    try:
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df
    except Exception as e:
        return e


In this section, we provide instructions on how to explore the simulated MES database using the provided code. The code demonstrates how to:

1. Connect to the SQLite database using the `sqlite3` library.
2. Execute SQL queries and retrieve the results as pandas DataFrames using the `query_sqlite` function.
3. Display the list of tables in the database using a SQL query.
4. Explore each table by retrieving the first 5 rows using SQL queries and storing the results in separate DataFrames.

The `query_sqlite` function is a utility function that takes a SQL query and the path to the SQLite database file as input. It establishes a connection to the database, executes the query using `pd.read_sql_query`, and returns the results as a pandas DataFrame. If an exception occurs, it returns the exception object.

You can modify the code below with your own queries to execute different SQL queries or explore the data in different ways. For example, you can change the SQL queries to filter, sort, or aggregate the data based on specific criteria. You can also join tables to retrieve related data from multiple tables.

Feel free to experiment with the code and explore the database further to gain insights into the simulated MES system.

You will notice how it can be difficult to write queries when you are not very familiar with the database structure or not familiar with the SQL language.

In [19]:
# Show tables in the database
print("Tables in the database:")
tables = query_sqlite("SELECT name FROM sqlite_master WHERE type='table';", "mes.db")
print(tables)

# Explore the Products table
print("\nProducts Table:")
products_df = query_sqlite("SELECT * FROM Products LIMIT 5;", "mes.db")
print(products_df)

# Explore the Machines table
print("\nMachines Table:")
machines_df = query_sqlite("SELECT * FROM Machines LIMIT 5;", "mes.db")
print(machines_df)

# Explore the Work Orders table
print("\nWork Orders Table:")
work_orders_df = query_sqlite("SELECT * FROM WorkOrders LIMIT 5;", "mes.db")
print(work_orders_df)

# Explore the Inventory table
print("\nInventory Table:")
inventory_df = query_sqlite("SELECT * FROM Inventory LIMIT 5;", "mes.db")
print(inventory_df)

# Explore the Quality Control table
print("\nQuality Control Table:")
quality_control_df = query_sqlite("SELECT * FROM QualityControl LIMIT 5;", "mes.db")
print(quality_control_df)

# Explore the Employees table
print("\nEmployees Table:")
employees_df = query_sqlite("SELECT * FROM Employees LIMIT 5;", "mes.db")
print(employees_df)

Tables in the database:
              name
0         Products
1  sqlite_sequence
2         Machines
3       WorkOrders
4        Inventory
5   QualityControl
6        Employees

Products Table:
   ProductID         Name                                        Description
0          1  Smart Phone  Sleek and stylish phone with high-resolution d...
1          2       Laptop  Thin and lightweight laptop with long battery ...
2          3       Tablet  Portable and versatile tablet with crisp displ...
3          4   Desktop PC  Powerful desktop computer with abundant storag...
4          5       Router  Robust wireless router with blazing-fast speed...

Machines Table:
   MachineID       Name    Type       Status
0          1  Machine 1  Type B      running
1          2  Machine 2  Type C         idle
2          3  Machine 3  Type C  maintenance
3          4  Machine 4  Type A      running
4          5  Machine 5  Type A  maintenance

Work Orders Table:
   OrderID  ProductID  Quantity   Star

## Using Generative AI to construct SQL queries
In order to write a query against our system of record, an MES here, the LLM must be provided information as to what type of system it is (MSSQL, PostgreSQL, SQLite, etc) so it know what query language to use. Additionally, prompt engineering is critical here to ensure that the LLM outputs a standard format that we will be able to parse later.

Different model families respond differently to formatting instructions. For this workshop, since we are leveraging Anthropic Claude models, we will section the different part of our queries (input) and response (output) using XML tags as it tends to provide the most consistent results.
If using a different model family, for example Mixtral, Cohere, Llama, you would need to update the prompt template to get the ideal result.

In [20]:
llm = get_llm(model_id="anthropic.claude-3-haiku-20240307-v1:0", model_kwargs={"max_tokens": 4096,"temperature": 0.1})

# For Claude 3 Haiku
response = llm.invoke("Generate a SQL query to answer the following against my MES: What is the status of each machine in the facility?").content
print(response)

To get the status of each machine in the facility, you can use the following SQL query:

```sql
SELECT
    m.machine_id,
    m.machine_name,
    m.machine_status
FROM
    machines m
```

Here's a breakdown of the query:

1. `SELECT m.machine_id, m.machine_name, m.machine_status`: This part of the query specifies the columns you want to retrieve from the `machines` table. In this case, we're selecting the `machine_id`, `machine_name`, and `machine_status` columns.

2. `FROM machines m`: This part of the query specifies the table you want to query, which is the `machines` table. The `m` alias is used to refer to the `machines` table throughout the query.

This query will return a result set with the following columns:

- `machine_id`: The unique identifier for each machine.
- `machine_name`: The name of each machine.
- `machine_status`: The current status of each machine.

The result set will have one row for each machine in the facility, with the corresponding machine information.

Note

Let's run the query! Paste the query returned by the LLM in the following cell:

In [21]:
query = """
SELECT
    m.machine_id,
    m.machine_name,
    m.status
FROM
    machines m
"""

query_sqlite(query=query, db_path="mes.db")

pandas.errors.DatabaseError("Execution failed on sql '\nSELECT\n    m.machine_id,\n    m.machine_name,\n    m.status\nFROM\n    machines m\n': no such column: m.machine_id")

Notice how it fails becomes it does not actually know what the tables are and how the data is stored in our MES.

Try to change the prompt in the cell above to get a query that is able to answer the question.

## Provide database information to the LLM
Let's add context to the prompt so the model can understand how and where the data is stored.

Langchain simplifies the use of SQL Alchemy to obtain information on the table names, their structure and some sample data.

This context in critical for the LLM to write queries that will return the expected result. This way, if a user asks for the **active** work orders, but they are stored as `in progress` the LLM will be able to reason what value to use and also understand what tables to access and their relationship.

This is similar to what we did during the exploration section. We need to know how the MES is structured before we can start writing SQL Queries. We will automate this step in the next section!

In [22]:
from langchain_community.utilities import SQLDatabase

def get_mes_schema(db_path):
    """
    Get the schema of the MES database.

    This function uses the SQLDatabase utility from the langchain_community library
    to retrieve the schema information of the MES database. It fetches the table names,
    their structures, and a few sample rows from each table.

    Parameters
    ----------
    db_path : str
        The path to the SQLite database file.

    Returns
    -------
    str
        The schema of the MES database as a string.
        The schema includes table names, column names, data types, and sample rows.
    """
    # Create an instance of the SQLDatabase utility
    # It takes the database URI (in this case, the SQLite file path)
    # and the number of sample rows to retrieve from each table (set to 8)
    db = SQLDatabase.from_uri(f"sqlite:///{db_path}", sample_rows_in_table_info=8)

    # Get the list of usable table names in the database
    # This excludes any system or internal tables
    tables = db.get_usable_table_names()

    # Retrieve the schema information for the specified tables
    # The get_table_info_no_throw method fetches the table structures and sample rows
    # It returns the schema as a formatted string
    schema = db.get_table_info_no_throw(tables)

    return schema

schema = get_mes_schema('mes.db')

print("MES Database Schema:")
print("---------------------")
print(schema)

MES Database Schema:
---------------------

CREATE TABLE "Employees" (
	"EmployeeID" INTEGER, 
	"Name" TEXT NOT NULL, 
	"Role" TEXT, 
	"Shift" TEXT, 
	PRIMARY KEY ("EmployeeID")
)

/*
8 rows from Employees table:
EmployeeID	Name	Role	Shift
1	Ronnie Suarez	Operator	night
2	Mrs. Tracy Wilson	Operator	evening
3	Robert Pearson	Operator	night
4	Melissa Estrada	Operator	night
5	Kimberly Johnson	Manager	evening
6	Courtney Collins	Technician	evening
7	Katherine Wilson	Technician	evening
8	David Wilson	Operator	night
*/


CREATE TABLE "Inventory" (
	"ItemID" INTEGER, 
	"Name" TEXT NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	"ReorderLevel" INTEGER NOT NULL, 
	PRIMARY KEY ("ItemID"), 
	CHECK (Quantity >= 0), 
	CHECK (ReorderLevel >= 0)
)

/*
8 rows from Inventory table:
ItemID	Name	Quantity	ReorderLevel
1	Nuts	523	70
2	Bolts	66	81
3	Screws	450	59
4	Wires	1000	60
5	Transistors	813	77
6	Resistors	931	92
7	Capacitors	969	49
8	Diodes	520	58
*/


CREATE TABLE "Machines" (
	"MachineID" INTEGER, 
	"Name"

In this section, we discussed the importance of providing the LLM with information about the database system and its structure in order to generate accurate SQL queries.

We started by invoking the LLM to generate a SQL query that answered the question "What is the status of each machine in the facility?" against our MES. However, without any additional context about the database schema, the LLM's generated query failed because it didn't know the table names and how the data is stored.

To address this issue, we introduced the `get_mes_schema` function that uses the SQLDatabase utility from the langchain_community library to retrieve the schema information of the MES database. This function fetches the table names, their structures, and a few sample rows from each table. The schema information is then printed in the prompt to provide an overview of the database structure.

By providing the LLM with this contextual information about the database schema, it can understand how and where the data is stored. This enables the LLM to write queries that access the correct tables and use the appropriate column names and values.

For example, if a user asks for the "active" work orders, but they are stored as "in progress" in the database, the LLM can reason about the value to use based on the provided schema information. It can also understand the relationships between tables and construct queries that join the necessary tables to retrieve the desired data.

In the next section, we will automate the process of providing the database schema information to the LLM to streamline the query generation process.

## Prompt Template
Let's create a prompt template where we can provide the schema and ask questions and add some clear instructions to the LLM to generate the output we want.

Note that when using Anthropic Claude models, it is recommended to leverage XML tags to segment the prompt and the desired output format. This makes both the instructions more robust and additionally allows us to easily parse the output for further processing

See [Use XML tags](https://docs.anthropic.com/claude/docs/use-xml-tags) for additional details
> XML tags are a powerful tool for structuring prompts and guiding Claude's responses. Claude is particularly familiar with prompts that have XML tags as Claude was exposed to such prompts during training. By wrapping key parts of your prompt (such as instructions, examples, or input data) in XML tags, you can help Claude better understand the context and generate more accurate outputs. This technique is especially useful when working with complex prompts or variable inputs.

XML tags are elements used to structure and mark up data. They are enclosed in angle brackets (<>) and provide a way to label and organize different parts of the data. They should always be used in pairs and never just as the first half of a set.

For example: 
```
 <context> part of my prompt </context>
 <schema> my database schema </schema>
 <tables> relevant tables and description </tables>
 <question> here is the user question to answer </question>
 <output> Provide your output to the question by generating a SQL query, using the schema in <schema></schema> and only querying the tables in <tables></tables>. Write your answer in the <answer></answer> tags</output> 
 ```


In [30]:
instructions = """Given the database description and schema in the <schema> tag, including the sample data, write a SQL statement to answer the question provided in the task tag.
The database engine is SQLite, so ensure that all functions are SQLite compatible.
Ensure any column or table names that contain special characters or start with numbers are wrapped in double quotes.
When filtering on string values, use the lower() function to match case-insensitively.
For filtering on other data types, refer to the sample data provided in the schema section to determine the correct values to use in the WHERE clause.
Cast all averages or rates to float to ensure proper calculation.
Any valid SQL code should be wrapped inside <sql></sql> tags.
If the question is not related to the provided database information, respond that the question is not relevant and wrap the response in <error></error> tags.
If the question is ambiguously worded, ask for clarification.
If the question cannot be answered with the tables present in the database, state as such."""

question = "What is the status of each machine in the facility?"

def generate_sql_prompt(question,
                        instructions,
                        db_path):
    """Generates an prompt for text-to-SQL. Currently tuned for Claude which
    leverages xml tagging to separate key parts of the context. Supplies the 
    model with table description, table name, current date, table schema, 
    sample data. Includes model specific instructions.
    Parameters
    ----------
    question :
        question to be answered
    instructions :
        Model specific instructions engineered for text-to-SQL
    db_path :
        Path to the SQLite database file
    Returns
    ----------
    str :
        Prompt for text-to-SQL generation
    """

    # add db description and schema
    sql_prompt = f"""<description>\n This database simulates a Manufacturing Execution System (MES), which is a software system designed to manage the production process of products. The MES is used to track the production process, maintain the inventory, and ensure the quality of the products. The MES is designed to be used in a manufacturing environment, where products are manufactured, machines are used to produce products, work orders are created and tracked, and quality control is performed.\n</description>\n\nThe database schema is as follows:"""
    schema = get_mes_schema(db_path=db_path)
    sql_prompt += f"\n\n<schema> {schema} \n</schema>\n\n"
    # add in user question and task instructions
    sql_prompt += instructions
    sql_prompt += "\n\nThe task is:"
    sql_prompt += f"\n<task>\n{question}\n</task>\n\n"

    return sql_prompt


sql_prompt = generate_sql_prompt(question=question, instructions=instructions, db_path='mes.db')
response = llm.invoke(sql_prompt).content

print("SQL Prompt:\n" + sql_prompt + "\n\n\n Response:\n" + response)

SQL Prompt:
<description>
 This database simulates a Manufacturing Execution System (MES), which is a software system designed to manage the production process of products. The MES is used to track the production process, maintain the inventory, and ensure the quality of the products. The MES is designed to be used in a manufacturing environment, where products are manufactured, machines are used to produce products, work orders are created and tracked, and quality control is performed.
</description>

The database schema is as follows:

<schema> 
CREATE TABLE "Employees" (
	"EmployeeID" INTEGER, 
	"Name" TEXT NOT NULL, 
	"Role" TEXT, 
	"Shift" TEXT, 
	PRIMARY KEY ("EmployeeID")
)

/*
8 rows from Employees table:
EmployeeID	Name	Role	Shift
1	Ronnie Suarez	Operator	night
2	Mrs. Tracy Wilson	Operator	evening
3	Robert Pearson	Operator	night
4	Melissa Estrada	Operator	night
5	Kimberly Johnson	Manager	evening
6	Courtney Collins	Technician	evening
7	Katherine Wilson	Technician	evening
8	Davi

OK, let's try this new query and see if we can finally answer the question from our DB

In [31]:
import re

def parse_generated_sql(response):
    """Given a text-to-SQL generated output, extract the provided SQL. If query
    cannot be extracted than return the full response.
    Parameters
    ----------
    response :
        text-to-SQL generated string
    Returns
    ----------
    str :
        either the SQL from the generated text or the original response
    """
    try:
        start_sql = re.search(r'<sql>', response).end()
        end_sql = re.search(r'</sql>', response).start()
        query = response[start_sql:end_sql].strip()
        return query, True
    except:
        return response, False

query, success = parse_generated_sql(response)
print("Question:\n" + question)
print("\n\nSQL Query:\n"+ query)
df = query_sqlite(query=query, db_path="mes.db")
print("\n\nResult set:\n" + df.to_string())



Question:
What is the status of each machine in the facility?


SQL Query:
SELECT 
  "Machines"."MachineID",
  "Machines"."Name",
  "Machines"."Type",
  "Machines"."Status"
FROM "Machines";


Result set:
   MachineID       Name    Type       Status
0          1  Machine 1  Type B      running
1          2  Machine 2  Type C         idle
2          3  Machine 3  Type C  maintenance
3          4  Machine 4  Type A      running
4          5  Machine 5  Type A  maintenance


Great, we were able to generate a valid query, asking the same question that failed earlier. We can now improve our prompt template some more by providing the current date and time. This will help the LLM provide better queries that deal with time, such as `"What work orders were completed this year?"`. Let's modify our prompt template to provide the current date time.


In [25]:
# Add time
from datetime import datetime 
def generate_sql_prompt(question,
                        instructions,
                        db_path,
                        current_date=datetime.strftime(datetime.now(), '%A, %Y-%m-%d')):
    """Generates an prompt for text-to-SQL. Currently tuned for Claude which
    leverages xml tagging to separate key parts of the context. Supplies the 
    model with table description, table name, current date, table schema, 
    sample data. Includes model specific instructions.
    Parameters
    ----------
    question :
        question to be answered
    instructions :
        Model specific instructions engineered for text-to-SQL
    db_path :
        Path to the SQLite database file
    current_date :
        A specified date. (default=datetime.now())
    Returns
    ----------
    str :
        Prompt for text-to-SQL generation
    """

    # begin the prompt with the current date
    sql_prompt = f"Current Date: {current_date}\n\n"
    # add db description and schema
    sql_prompt += f"""<description>\n This database simulates a Manufacturing Execution System (MES), which is a software system designed to manage the production process of products. The MES is used to track the production process, maintain the inventory, and ensure the quality of the products. The MES is designed to be used in a manufacturing environment, where products are manufactured, machines are used to produce products, work orders are created and tracked, and quality control is performed.\n</description>\n\nThe database schema is as follows:"""
    schema = get_mes_schema(db_path=db_path)
    sql_prompt += f"\n\n<schema> {schema} \n</schema>\n\n"
    # add in user question and task instructions
    sql_prompt += instructions
    sql_prompt += "\n\nThe task is:"
    sql_prompt += f"\n<task>\n{question}\n</task>\n\n"

    return sql_prompt

question = "What work orders were completed this year?"

sql_prompt = generate_sql_prompt(question=question, instructions=instructions, db_path='mes.db')
response = llm.invoke(sql_prompt).content
query, success = parse_generated_sql(response)
print("SQL Query:\n"+ query)
df = query_sqlite(query=query, db_path="mes.db")
print("\n\nResult set:\n" + df.to_string())

SQL Query:
SELECT 
  "OrderID",
  "ProductID",
  "Quantity",
  "StartDate",
  "EndDate",
  "Status"
FROM "WorkOrders"
WHERE "Status" = 'completed'
  AND "EndDate" BETWEEN '2024-01-01' AND '2024-12-31';


Result set:
   OrderID  ProductID  Quantity   StartDate     EndDate     Status
0        2          9        62  2023-08-02  2024-06-23  completed
1       11         42        13  2023-10-23  2024-05-25  completed
2       13         48        82  2023-03-21  2024-12-23  completed
3       19         43        67  2023-10-28  2024-06-24  completed


## NLP Response
We have successfully implemented functions that allow us to ask questions against the MES and for the LLM to provide valid SQL queries that we can parse and execute against our MES. To provide this functionality in a chatbot, we not only have to get the result set, but use this dataframe to answer the original question from the user.

Let us now create a new prompt template to answer the original question with the additional dataframe coming from the MES as additional context.

In [26]:
nlp_instructions = """Answer the question in the <task></task> tags using the data provided by the SQL output in the <data></data> tags.
If the provided response is an empty table write back that the query returned no results.
The answer should be put inside <response></response> tags.
The response should be clear and human-readable (lists be separated by commas) and make sense for an Engineer.
Use markdown format as appropriate.
If you used the results in <data> state that the data came directly from the MES"""


def generate_nlp_prompt(data, question, query, instructions):
    """Generates a prompt given the results of the text-to-SQL request
    Parameters
    ----------
    data :
        the data returned from the database
    question :
        the original question asked
    query :
        the query used to return the data
    instructions :
        engineered instructions that are specific to the model provided
    Returns
    ----------
    str :
        An engineered prompt for summarizing the SQL results
    """
    nlp_prompt = """<task>\n%s\n</task>\n\n<sql>%s</sql>\n\n<data>\n%s\n</data>\n\n%s"""

    prompt = nlp_prompt % (question, query, data, instructions)

    return prompt

nlp_prompt = generate_nlp_prompt(df, question, query, nlp_instructions)
print("NLP Prompt")
print("-----------------")
print(nlp_prompt)



NLP Prompt
-----------------
<task>
What work orders were completed this year?
</task>

<sql>SELECT 
  "OrderID",
  "ProductID",
  "Quantity",
  "StartDate",
  "EndDate",
  "Status"
FROM "WorkOrders"
WHERE "Status" = 'completed'
  AND "EndDate" BETWEEN '2024-01-01' AND '2024-12-31';</sql>

<data>
   OrderID  ProductID  Quantity   StartDate     EndDate     Status
0        2          9        62  2023-08-02  2024-06-23  completed
1       11         42        13  2023-10-23  2024-05-25  completed
2       13         48        82  2023-03-21  2024-12-23  completed
3       19         43        67  2023-10-28  2024-06-24  completed
</data>

Answer the question in the <task></task> tags using the data provided by the SQL output in the <data></data> tags.
If the provided response is an empty table write back that the query returned no results.
The answer should be put inside <response></response> tags.
The response should be clear and human-readable (lists be separated by commas) and make sense

We can now invoke the LLM with this prompt and parse the output

In [27]:
def parse_generated_nlp(response):
    """Extract the response from the model. Currently built for Claude XML 
    tagging format
    Parameters
    ----------
    response :
        LLM generated string
    Returns
    ----------
    str :
        extracted string
    """
    try:
        start = re.search(r'<response>', response).end()
        end = re.search(r'</response>', response).start()
        response = response[start:end].strip()
        return response
    except:
        return response
    
response = llm.invoke(nlp_prompt).content
parsed_response = parse_generated_nlp(response)

print("QUESTION:\n" + question)
print("\n\nRESPONSE:\n" + parsed_response)
print("\n\nQUERY:\n" + query)


QUESTION:
What work orders were completed this year?


RESPONSE:
Based on the data provided in the SQL output, the work orders that were completed this year are:

2, 11, 13, 19

The data came directly from the MES.


QUERY:
SELECT 
  "OrderID",
  "ProductID",
  "Quantity",
  "StartDate",
  "EndDate",
  "Status"
FROM "WorkOrders"
WHERE "Status" = 'completed'
  AND "EndDate" BETWEEN '2024-01-01' AND '2024-12-31';


In this section, we focused on creating a prompt template that included the database schema, sample data, and clear instructions for the LLM to generate the desired SQL queries. The `generate_sql_prompt` function took the question, instructions, and database path as input and constructed the prompt by combining the database description, schema, and the task instructions.

We then tested the generated SQL query and saw that it successfully answered the question that failed earlier. However, we noticed that the LLM was trying its best but not providing a valid query that respected the meaning of the question, specifically when dealing with dates.

To address this, we modified the `generate_sql_prompt` function to include the current date and time in the prompt. This additional context helped the LLM generate better queries that handled time-related conditions more accurately.

Next, we moved on to generating an NLP response based on the result set obtained from the SQL query. We created a new prompt template that included the original question, the SQL query used, and the result set as additional context. The `generate_nlp_prompt` function constructed this prompt, which was then used to invoke the LLM and generate a human-readable response.

Finally, we parsed the generated NLP response using the `parse_generated_nlp` function, which extracted the relevant content from the LLM's output based on the XML tagging format.

## Let's Put This All Together
Let's bring everything together and test various questions!

In [34]:
question = "who are the workers on night shift"

start_time = time.time()
sql_prompt = generate_sql_prompt(question=question, instructions=instructions, db_path='mes.db')
response = llm.invoke(sql_prompt).content
query, success = parse_generated_sql(response)
df = query_sqlite(query=query, db_path="mes.db")
nlp_prompt = generate_nlp_prompt(df, question, query, nlp_instructions)
response = llm.invoke(nlp_prompt).content
parsed_response = parse_generated_nlp(response)
end_time = time.time()
execution_time = end_time - start_time

print("QUESTION:\n" + question)
print("\n\nRESPONSE:\n" + parsed_response)
print("\n\nQUERY:\n" + query)
print("\n\nRESULT SET:\n" + df.to_string())
print(f"\n\nTotal execution time: {execution_time2:.2f} seconds")


QUESTION:
who are the workers on night shift


RESPONSE:
The workers on the night shift, based on the data provided in the SQL output, are:

Ronnie Suarez (Operator), Robert Pearson (Operator), Melissa Estrada (Operator), David Wilson (Operator), and Amy Davis (Operator).

The data for this response came directly from the MES.


QUERY:
SELECT "Name", "Role"
FROM "Employees"
WHERE "Shift" = 'night';


RESULT SET:
              Name      Role
0    Ronnie Suarez  Operator
1   Robert Pearson  Operator
2  Melissa Estrada  Operator
3     David Wilson  Operator
4        Amy Davis  Operator


Total execution time: 4.66 seconds


## Conclusion

This concludes the workshop, where we successfully implemented functions to ask questions against the MES, generate valid SQL queries using the LLM, execute those queries against the database, and generate human-readable responses based on the result set. Throughout the workshop, we focused on creating a prompt template that included the database schema, sample data, and clear instructions for the LLM to generate the desired SQL queries. We also addressed challenges related to time-based queries by including the current date and time in the prompt. Finally, we created a new prompt template to generate NLP responses based on the SQL query results, providing a more user-friendly output for the chatbot functionality.